<a href="https://colab.research.google.com/github/akashban812/DS-practice/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os as os
os.chdir('/content/drive/MyDrive/DS notebooks')
os.getcwd()
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [12]:
#@title Default title text
##Importing the data
##Unicode escape in activated to encounter any error arising due to any invalid characters

## Using the 'Online Retail' dataset from Kaggle


rtl_raw_data=pd.read_csv('OnlineRetail.csv',encoding='unicode_escape')



In [13]:
###checking distribution by country
country_cust=rtl_raw_data[['Country','CustomerID']].drop_duplicates()
country_cust_grp= country_cust.groupby(['Country'])['CustomerID'].nunique().reset_index().sort_values(by=['CustomerID'],ascending=[False])
country_cust_grp

##In the next step we'll use only the records for UK as it has the max customers
rtl_raw_UK=rtl_raw_data[rtl_raw_data['Country']=='United Kingdom'].reset_index(drop=True)

In [14]:

##Checking for missng values
rtl_raw_UK.isnull().sum(axis=0)



InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [15]:
##removing missing values for the customerid column
rtl_raw_UK_2=rtl_raw_UK[pd.notnull(rtl_raw_UK['CustomerID'])]

##Validating if there are any negative vales in the Quantity column
rtl_raw_UK_2.Quantity.min()


-80995

In [16]:
##Validating if there are any negative vales in the UnitPrice column
rtl_raw_UK_2.UnitPrice.min()

0.0

In [17]:
##Filtering out negative values from Quantity column
rtl_raw_UK_3=rtl_raw_UK_2[rtl_raw_UK_2['Quantity']>0]

##Converting the date filed to datetime
rtl_raw_UK_3['InvoiceDate']=pd.to_datetime(rtl_raw_UK_3['InvoiceDate'])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
###Calculation of total amt
rtl_raw_UK_3['total_spend']=rtl_raw_UK_3['Quantity']*rtl_raw_UK_3['UnitPrice']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### From this point onwards, we demonstrate RFM modeling using RFM scores


In [30]:
import datetime as dt
### Setting latest date as per dataset
latest_dt=dt.datetime(2011,12,10)

###Creating RFM scores for each customer

rfm_scores= rtl_raw_UK_3.groupby(['CustomerID']).agg({'InvoiceDate': lambda x: (latest_dt - x.max()).days,\
                                                      'InvoiceNo': lambda x: len(x), 'total_spend': lambda x: x.sum()}).reset_index()

rfm_scores.rename(columns={'InvoiceDate':'recency','InvoiceNo':'frequency','total_spend':'monetary'},inplace=True)
rfm_scores

,CustomerID,recency,frequency,monetary
0,12346.0,325,1,77183.60
1,12747.0,22,103,4196.01
2,12748.0,4,4596,33719.73
3,12749.0,22,199,4090.88
4,12820.0,44,59,942.34
...,...,...,...,...
3916,18280.0,159,10,180.60
3917,18281.0,3,7,80.82
3918,18282.0,215,12,178.05
3919,18283.0,9,756,2094.88


In [32]:
qcols=['recency','frequency','monetary']


for i in qcols:
  qnt=rfm_scores[i].quantile(q=[0.25,0.5,0.75])
  print(qnt)

0.25     21.0
0.50     60.0
0.75    161.0
Name: recency, dtype: float64
0.25    17.0
0.50    41.0
0.75    99.0
Name: frequency, dtype: float64
0.25     300.04
0.50     651.82
0.75    1575.89
Name: monetary, dtype: float64
